In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import key_dict

In [23]:
bitcoin = pd.read_csv('bitcoin all time.csv')
dow_jones = pd.read_csv('HistoricalDJIA.csv')

In [25]:
bitcoin = bitcoin[['Date', 'Closing Price (USD)']]
dow_jones = dow_jones[['Date', ' Close']]

bitcoin.Date = pd.to_datetime(bitcoin.Date)
dow_jones.Date = pd.to_datetime(dow_jones.Date)

bitcoin.reset_index(inplace = True, drop = True)
dow_jones.reset_index(inplace = True, drop = True)

bitcoin['id'] = list(bitcoin.index.values)
dow_jones['id'] = list(dow_jones.index.values)

In [33]:
bitcoin.rename(columns = {"Date":"date_", "Closing Price (USD)":"price_usd_bc"}, inplace = True)
dow_jones.rename(columns = {"Date":"date_", " Close":"score"}, inplace = True)

In [8]:
connection_string = "postgres:" + key_dict['postgres'] + "@localhost:5432/etl-project"
engine = create_engine(f"postgresql://{connection_string}")

In [10]:
engine.table_names()

F:\anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['gold_data', 'bitcoin_data', 'djia_data']

In [37]:
bitcoin.to_sql(name = 'bitcoin_data', con = engine, if_exists = 'replace', index = False)
dow_jones.to_sql(name = 'djia_data', con = engine, if_exists = 'replace', index = False)

In [39]:
pd.read_sql_query("SELECT bitcoin_data.date_, bitcoin_data.price_usd_bc, djia_data.score \
                    FROM bitcoin_data \
                    JOIN djia_data \
                    ON bitcoin_data.date_ = djia_data.date_", con = engine)

,date_,price_usd_bc,score
0,2013-09-30,123.654990,15129.67
1,2013-10-01,125.455000,15191.70
2,2013-10-02,108.584830,15133.14
3,2013-10-03,118.674660,14996.48
4,2013-10-04,121.338660,15072.58
...,...,...,...
1906,2021-04-23,50669.144382,34043.49
1907,2021-04-26,53558.707845,33981.57
1908,2021-04-27,55123.861981,33984.93
1909,2021-04-28,54591.515326,33820.38
